[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PyMPDATA/blob/main/examples/PyMPDATA_examples/Magnuszewski_et_al_2025/table_1.ipynb)
[![launch on mybinder.org](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PyMPDATA.git/main?urlpath=lab/tree/examples/PyMPDATA_examples/Magnuszewski_et_al_2025/table_1.ipynb)
[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PyMPDATA/blob/main/examples/PyMPDATA_examples/Magnuszewski_et_al_2025/table_1.ipynb)

## Paweł Magnuszewski MSc project

Tamble comparing herein computed UPWIND, MPDATA and Monte-Carlo solutions with data from literature

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PyMPDATA-examples')

In [ ]:
import os
import time

import matplotlib.pyplot as plt
import pandas
import numpy as np
from matplotlib import pyplot
import pandas as pd
from ipywidgets import IntProgress

from PyMPDATA_examples.Magnuszewski_et_al_2025.asian_option import AsianArithmetic, Settings
from PyMPDATA_examples.Magnuszewski_et_al_2025.common import OPTIONS
from PyMPDATA_examples.Magnuszewski_et_al_2025.monte_carlo import BSModel, FixedStrikeGeometricAsianOption, FixedStrikeArithmeticAsianOption
import PyMPDATA_examples.utils.financial_formulae.asian_option as asian_analytic
from PyMPDATA_examples.Magnuszewski_et_al_2025 import barraquand_data

pandas.options.display.float_format = '{:,.3f}'.format

In [ ]:
CI = 'CI' in os.environ

s_min = 50
s_max = 200

mc_n_paths = [1000, 10000] if not CI else [10, 100]
mc_seed = 42
mc_path_points = 1000 if not CI else 10

spot = 100
risk_free_rate = 0.1

In [ ]:
def run_numeric_and_mc(params, *, nx, ny, nt, variant):
    settings = Settings(**params, r=risk_free_rate, S_max=s_max, S_min=s_min)
    mc_model = BSModel(T=params['T'],
                       sigma=params['sgma'],
                       r=risk_free_rate,
                       M=mc_path_points,
                       S0=spot,
                       seed=mc_seed)
    simulations = {k:AsianArithmetic(settings, nx=nx, ny=ny, nt=nt, variant=variant, options=OPTIONS[k]) for k in OPTIONS}
    results = {}
    
    for k, simulation in simulations.items():
        simulation.step(simulation.nt)
        simulation_price = simulation.solver.advectee.get()[:, 0]
        results[k] = np.interp(spot, simulation.S, simulation_price)
    for mc_n_path in mc_n_paths:
        arithmetic_option = FixedStrikeArithmeticAsianOption(params['T'], params['K'], variant, mc_model, mc_n_path)
        results[f"MC_{mc_n_path}"] = arithmetic_option.price_by_mc()
    return results

In [ ]:
discretization_parameters = {
    (.1,.25, 95):{'nx': 201, 'ny': 400, 'nt': 1500},
    (.1,.25,100):{'nx': 201, 'ny': 400, 'nt': 1500},
    (.1,.25,105):{'nx': 201, 'ny': 400, 'nt': 1500},
    (.1,.5,  95):{'nx': 201, 'ny': 300, 'nt': 1500},
    (.1,.5, 100):{'nx': 201, 'ny': 300, 'nt': 1500},
    (.1,.5, 105):{'nx': 201, 'ny': 300, 'nt': 1500},
    (.1,1.,  95):{'nx': 251, 'ny': 250, 'nt': 1500},
    (.1,1., 100):{'nx': 251, 'ny': 250, 'nt': 1500},
    (.1,1., 105):{'nx': 251, 'ny': 250, 'nt': 1500},
    (.2,.25, 95):{'nx': 251, 'ny': 250, 'nt': 1000},
    (.2,.25,100):{'nx': 251, 'ny': 250, 'nt': 1000},
    (.2,.25,105):{'nx': 251, 'ny': 250, 'nt': 1000},
    (.2,.5,  95):{'nx': 101, 'ny': 200, 'nt': 1000},
    (.2,.5, 100):{'nx': 101, 'ny': 200, 'nt': 1000},
    (.2,.5, 105):{'nx': 101, 'ny': 200, 'nt': 1000},
    (.2,1.,  95):{'nx': 101, 'ny': 200, 'nt': 1500},
    (.2,1., 100):{'nx': 101, 'ny': 200, 'nt': 1500},
    (.2,1., 105):{'nx': 101, 'ny': 200, 'nt': 1200},
    (.4,.25, 95):{'nx': 101, 'ny': 110, 'nt': 2200},
    (.4,.25,100):{'nx': 101, 'ny': 200, 'nt': 1000},
    (.4,.25,105):{'nx': 101, 'ny': 200, 'nt': 1500},
    (.4,.5,  95):{'nx': 101, 'ny': 110, 'nt': 1000},
    (.4,.5, 100):{'nx': 101, 'ny': 110, 'nt': 1000},
    (.4,.5, 105):{'nx': 101, 'ny': 110, 'nt': 1000},
    (.4,1.,  95):{'nx': 101, 'ny': 100, 'nt': 1800},
    (.4,1., 100):{'nx': 101, 'ny': 100, 'nt': 1780},
    (.4,1., 105):{'nx': 101, 'ny': 100, 'nt': 1760},
} if not CI else {
    (.1,.25, 95):{'nx': 21, 'ny': 40, 'nt': 40},
    (.1,.25,100):{'nx': 21, 'ny': 40, 'nt': 40},
}

In [ ]:
barraquand_df = pd.DataFrame(columns=barraquand_data.headers)
for line in barraquand_data.table.strip('\n').split('\n'):
    data_row = line.split(',')
    if len(data_row) > 0:
        barraquand_df.loc[len(barraquand_df)] = data_row
barraquand_df['call_price'] = barraquand_df['call_price'].astype(float)
barraquand_df['put_price'] = barraquand_df['put_price'].astype(float)

In [ ]:
def calculate_row(row_idx):
    row_data = barraquand_df.iloc[row_idx].astype(float)
    nx,ny,nt = discretization_parameters[(row_data['sigma'], row_data['T'], row_data['K'])].values()
    simulation_params = {
        'sgma':row_data['sigma'],
        'T':row_data['T'],
        'K':row_data['K']
    }
    call_price = row_data['call_price']
    put_price = row_data['put_price']
    results_call = run_numeric_and_mc(simulation_params, nx=nx, ny=ny, nt=nt, variant='call')
    results_put = run_numeric_and_mc(simulation_params, nx=nx, ny=ny, nt=nt, variant='put')
    return (
        {k: round(v,3) for k, v in results_call.items()},
        {k: round(v,3) for k, v in results_put.items()},
        simulation_params, call_price, put_price
    )

In [ ]:
results_df = pd.DataFrame(columns=[
    'sigma', 'T', 'K',
    'BP_call',
    'UPWIND_call', 'MPDATA_call',
    f'MC_{mc_n_paths[0]}_call', f'MC_{mc_n_paths[1]}_call',
    'BP_put', 'UPWIND_put',
    'MPDATA_put',
    f'MC_{mc_n_paths[0]}_put', f'MC_{mc_n_paths[1]}_put'
])
progbar = IntProgress(max=len(discretization_parameters))
display(progbar)
for i, _ in enumerate(discretization_parameters):
    call, put, params, call_bp, put_bp = calculate_row(i)
    new_row = [*params.values(), call_bp, *call.values(), put_bp, *put.values()]
    results_df.loc[len(results_df)] = new_row
    progbar.value += 1
results_df['K'] = results_df['K'].astype(int)
display(results_df)